Installing the dependencies

In [2]:
!python --version

Python 3.10.12


In [5]:
!pip install fastapi
!pip install uvicorn
!pip install pickle5
!pip install pydantic
!pip install scikit-learn
!pip install requests
!pip install pypi-json
!pip install pyngrok
!pip install nest-asyncio
!pip install transformers
!pip install torch
!pip install IPython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 40.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.0 MB/s eta 0:00:00


In [14]:
from fastapi import FastAPI
from pydantic import BaseModel, Field
import pickle
import json
import uvicorn
from pyngrok import ngrok
from fastapi.middleware.cors import CORSMiddleware
import nest_asyncio
from typing_extensions import Literal
from transformers import VitsModel, VitsConfig, AutoTokenizer
import torch
from IPython.display import Audio
from base64 import b64encode, b64decode
import pandas as pd

In [7]:
app = FastAPI()

In [8]:
origins = ["*"]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

model = VitsModel.from_pretrained("facebook/mms-tts-rus")
tokenizer = AutoTokenizer.from_pretrained("facebook/mms-tts-rus")

Some weights of the model checkpoint at facebook/mms-tts-rus were not used when initializing VitsModel: ['flow.flows.0.wavenet.res_skip_layers.3.weight_v', 'flow.flows.0.wavenet.res_skip_layers.2.weight_g', 'flow.flows.1.wavenet.res_skip_layers.1.weight_g', 'flow.flows.3.wavenet.res_skip_layers.2.weight_v', 'posterior_encoder.wavenet.in_layers.12.weight_g', 'flow.flows.2.wavenet.res_skip_layers.3.weight_v', 'flow.flows.1.wavenet.in_layers.0.weight_g', 'flow.flows.1.wavenet.res_skip_layers.1.weight_v', 'posterior_encoder.wavenet.res_skip_layers.0.weight_v', 'flow.flows.0.wavenet.res_skip_layers.0.weight_v', 'posterior_encoder.wavenet.in_layers.6.weight_g', 'flow.flows.2.wavenet.in_layers.3.weight_v', 'posterior_encoder.wavenet.res_skip_layers.6.weight_g', 'posterior_encoder.wavenet.in_layers.14.weight_v', 'posterior_encoder.wavenet.in_layers.4.weight_g', 'posterior_encoder.wavenet.in_layers.8.weight_v', 'flow.flows.0.wavenet.res_skip_layers.0.weight_g', 'posterior_encoder.wavenet.in_lay

In [9]:
class BreastData(BaseModel):
    text: str = Field(default='машина')

    class Config:
        # example data
        schema_extra = {
            "text": 'машина',
        }

class BreastPrediction(BaseModel):
    prediction: str

model_name = "Breast Cancer Wisconsin (Diagnostic)"
version = "v1.0.0"

In [10]:
def Text2Speech(text):
  inputs = tokenizer(text, return_tensors="pt")

  with torch.no_grad():
    output = model(**inputs).waveform

  audio = Audio(data = output, rate = model.config.sampling_rate)
  Fjson = b64encode(audio.data).decode('ascii')

  return Fjson


In [11]:
def get_model_response(text):
  text = pd.json_normalize(text.__dict__)["text"]
  inputs = tokenizer(text.values[0], return_tensors="pt")
  with torch.no_grad():
    output = model(**inputs).waveform

  audio = Audio(data = output, rate = model.config.sampling_rate)
  Fjson = b64encode(audio.data).decode('ascii')
  return {"prediction": Fjson}

In [12]:
@app.get("/info")
async def model_info():
    """Return model information, version, how to call"""
    return {"name": 'TTS', "version": 'latest'}

@app.get("/health")
async def service_health():
    """Return service health"""
    return {"ok"}

@app.post("/predict", response_model=BreastPrediction)
async def model_predict(sample: BreastData):
    """Predict with input"""
    response = get_model_response(sample)
    return response

In [ ]:
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

Public URL: https://fcdf-35-227-82-26.ngrok.io


INFO:     Started server process [273]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     149.154.161.247:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     149.154.161.252:0 - "GET /a HTTP/1.1" 404 Not Found
INFO:     149.154.161.212:0 - "GET /pre HTTP/1.1" 404 Not Found
INFO:     149.154.161.220:0 - "GET /pred HTTP/1.1" 404 Not Found
INFO:     149.154.161.234:0 - "GET /predict HTTP/1.1" 405 Method Not Allowed
INFO:     188.234.32.40:0 - "GET /predict HTTP/1.1" 405 Method Not Allowed
INFO:     188.234.32.40:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     188.234.32.40:0 - "GET /predict HTTP/1.1" 405 Method Not Allowed
INFO:     188.234.32.40:0 - "GET /predict/%22Content-Type%3A%20application/json%22%20-d%20%27%7B%20%22text%22%3A%20%22%D0%BC%D0%B0%D1%88%D0%B8%D0%BD%D0%B0%D0%B0%D0%B0%D0%B0%D0%B0%D0%B0%D0%B0%D0%B0%22%20%7D%27 HTTP/1.1" 404 Not Found
INFO:     188.234.32.40:0 - "GET /predict/%22Content-Type%3A%20application/json%22%20-d%20%27%7B%20%22text%22%3A%20%22%D0%BC%D0%B0%D1%88%D0%B8%D0%BD%D0%B0%D0%B0%D0%B0%D0%B0%D0%B0%D0%B0%D0%B0%D0%B0%22%20%7D%27 HTTP/1.1"